In [2]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
import copy

setup geometry

In [3]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "in"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.1))
mesh.Curve(3);
mesh_orig = copy.deepcopy(mesh)
#Draw(mesh)

setup FEM space

In [4]:
# viscosity
nu = 0.01
# Order of spaces
k = 2
# H1 vs VectorH1 -> vector field?!
V = VectorH1(mesh,order=k, dirichlet="top|bot|cyl|in|out")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q]) # X = [V,V,Q] (without VectorH1)

setup bilinear form
velocityfield u and pressurefield p

In [5]:
u,p = X.TrialFunction()
v,q = X.TestFunction()
# (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a = BilinearForm(X)
a += (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a.Assemble()

setup boundary conditions

In [6]:
gfu = GridFunction(X)
#'gfp = GridFunction(X)
# setup flow condition
uinf = 0.001
uin = CoefficientFunction((uinf,0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("in|top|bot|out"))

velocity = CoefficientFunction(gfu.components[0])
scene_state = Draw(velocity, mesh, "vel")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

solve equation

In [7]:
def solveStokes():
    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(X.FreeDofs())
    gfu.vec.data += inv * res
    scene_state.Redraw()
solveStokes()

### creation of gfset
function that will perturb our mesh

In [8]:
# Test and trial functions for shape derivate -> do we even need this?
VEC = H1(mesh, order=2, dim=2, dirichlet="top|bot|in|out")
PHI, PSI = VEC.TnT()
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
mesh.SetDeformation(gfset)
sceneSet = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

# deformation calculation?!
gfX = GridFunction(VEC)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

Drag/ "dissipated energy" [should be working]

$J(\Omega ) = \frac{1}{2} \int_\Omega Du : Du dx$ 

In [9]:
solveStokes()
vel = gfu.components[0] #gfu.vec.data
drag = Integrate(0.5*InnerProduct(Grad(vel),Grad(vel)),mesh)
print(drag)

# 1.0997749905576028e-05 - r=0.5
# 0.00043084429187575745 - r=1.5

1.0989139295909543e-05


In [10]:
def calc_drag(gfu):
    vel = gfu.components[0]
    return 0.5*InnerProduct(grad(vel),grad(vel))*dx

### get surface area of mesh [done]
(without ball), should stay constant

In [11]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

def calc_surf():
    return surf_t*dx-surf_0

def calc_surf_inv():
    return 1/(surf_t*dx)

def calc_surf_diff():
    return (surf_0 - surf_t*dx)**2

1/Integrate(surf_t,mesh)

0.04307633755446834

### try for barycenter <br>
$vol(\Omega) = \int_{\Omega} 1 \,dx \in \mathbb{R}$ <br>
von Hand abeleiten

$bc^\Omega = \frac{1}{vol(\Omega)}\int_{\Omega}x\,dx \in \mathbb{R}^d$


In [12]:
#real calculation:
print((1/Integrate(1,mesh))*Integrate(x,mesh))
print((1/Integrate(1,mesh))*Integrate(y,mesh))

-2.3687994282703518e-17
2.3800082583725774e-17


In [13]:
bc_t = CoefficientFunction(x)
bc_0 = 1/surf_0*Integrate(bc_t,mesh)
print(bc_0)
to = 1/CoefficientFunction(1)*CoefficientFunction(x)
Integrate(to,mesh)

-2.3687994282703518e-17


-5.499073418846478e-16

In [14]:
bc_t = CoefficientFunction(x)
bc_0 = 1/surf_0*Integrate(bc_t,mesh)

def calc_bc():
    return (calc_surf_inv()*(bc_t*dx))# - bc_0
    #return bc_t*dx
    
def calc_bc_test():
    return 1/CoefficientFunction(1)*dx*CoefficientFunction(x+y)*dx # does not work

In [15]:
n = specialcf.normal(2)
def DiffCost(gfu):
    #return 100*div(gfset)**2*dx
    return (CoefficientFunction((x,y))*n)**2*ds(definedon="cyl")

def Cost(u):
    return (CoefficientFunction((x,y))*n)**2*ds(definedon="cyl")

def Volume():
    return Integrate((CoefficientFunction((x,y))*n)**2*ds(definedon="cyl"),mesh)


## Adjoint equation try?
maybe we dont even need this??

* J hängt von u ab, u von omega
* dJ nach u -> "in Vektorfeldrichtung" -> für jedes Vektorfeld ableiten
* stattdessen adjoint -> 1x u und 1x omega
* adjoint
* Lagrangian du -> adjoint

In [16]:
gfp = GridFunction(X)
r, w = X.TnT()
fadjoint = LinearForm(X)
#fadjoint += -1*(Cost(gfu)).Diff(gfu,w)
# add by hand derived formula for constant volume:
fadjoint += -1*(DiffCost(gfu))

In [17]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(X.FreeDofs()).T * rhs
    gfp.vec.data += update

In [18]:
fadjoint.Assemble()
SolveAdjointEquation()
adjoint_sol = CoefficientFunction(gfp.components[0])
Draw (adjoint_sol, mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

### Shape derivative $\mathrm{d}J(\Omega)(\mathbf{X}) = \int_{\Omega} \mathbf{S}_1:D \mathbf{X} dx$
$S_1 = \biggl(\frac{1}{2}Du : Du - p div(u) \biggr) I_2 + Du^T p - Du^T Du.$ <br>
$\mathrm{u}$ and $p$ are the solutions to the stokesflow problem<br>
we use u = gfu[0], p = gfu[1]

# Auto shapediff try

In [19]:
def J(u):
    return 0.5*(InnerProduct(grad(u.components[0]),grad(u.components[0])))*dx # cost
    #return (InnerProduct(grad(u.components[0]),grad(v.components[0]))+div(u.components[0])*v.components[1]+div(v.components[0])*u.components[1])*dx # stokes
    #return (0.5*(InnerProduct(grad(u.components[0]),grad(u.components[0]))-v*div(u))*Id(2)+grad(u).trans*v-grad(u).trans*grad(u))*dx # shape derivative

Lagrangian = J(gfu)  #DiffCost(gfu.components[0])  + Cost(gfu)

In [20]:
dJOmega = LinearForm(VEC)
dJOmega += Lagrangian.DiffShape(PSI)
#dJOmega += 2*(Volume()-pi/4)*InnerProduct(n,PSI)*ds(definedon="cyl") #dVolume
#dJOmega += Equation(PHI,PSI)
#dJOmega += DiffCost.DiffShape(PSI)

b = BilinearForm(VEC)
b += InnerProduct(grad(PSI),grad(PHI))*dx + InnerProduct(PSI,PHI)*dx

# gfX = GridFunction(VEC)

## this basically just copy paste

In [21]:
def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [22]:
b.Assemble()
dJOmega.Assemble()
SolveDeformationEquation()
Draw(-gfX, mesh, "-gfX")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [23]:
print('Cost at initial design', Integrate (Cost(gfu), mesh))
gfset.Set((0,0))
scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec

Cost at initial design 0.7853990296189934


In [24]:
test_scene = Draw(gfset)
mesh.SetDeformation(gfset)
test_scene.Redraw()

a.Assemble()
fadjoint.Assemble()
solveStokes()
print('Cost at new design', Integrate (Cost(gfu), mesh))

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

Cost at new design 0.5774458352662157


In [25]:
#reset the design
gfset.Set((0,0))
mesh.SetDeformation(gfset)

#check if initial value of cost function 0.000486578350214552 is recovered
a.Assemble()
fadjoint.Assemble()
solveStokes()
print('Cost at new design', Integrate (Cost(gfu), mesh))

Cost at new design 0.7853990296189934


# general idea: of iteration

* solve solution
* deform
* calc cost after deform
* look how cost changed -> armijo rule
* abstieg: -gradient²

In [27]:
scene = Draw(gfset)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [28]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
a.Assemble()
fadjoint.Assemble()
solveStokes()

LineSearch = False

iter_max = 20
Jold = Integrate(Cost(gfu), mesh)
converged = False

# try parts of loop
mesh.SetDeformation(gfset)
scene.Redraw()

print('cost at iteration', k, ': ', Jold)
    
# input("Press enter to start optimization")
for k in range(iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    scene_state.Redraw()
    
    print('cost at iteration', k, ': ', Jold)
    print('volume iteration', k, ': ', Volume())
    a.Assemble()
    solveStokes()
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmega.Assemble()
    SolveDeformationEquation()

    #scale = 0.01 / Norm(gfX.vec)
    scale = 0.1 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    Jnew = Integrate(DiffCost(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            #input('a')
            scale = scale / 2
            print("scale = ", scale)
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            a.Assemble()
            solveStokes()
            Jnew = Integrate(Cost(gfu), mesh)
    
    if converged==True:
        print("No more descent can be found")
        break
    Jold = Jnew

    Redraw(blocking=True)

cost at iteration 2 :  0.7853990296189934
cost at iteration 0 :  0.7853990296189934
volume iteration 0 :  0.7853990296189934
cost at iteration 1 :  0.740093782438725
volume iteration 1 :  0.740093782438725
cost at iteration 2 :  0.6965783012250752
volume iteration 2 :  0.6965783012250752
cost at iteration 3 :  0.6549585275936112
volume iteration 3 :  0.6549585275936112
cost at iteration 4 :  0.6153230729682527
volume iteration 4 :  0.6153230729682527
cost at iteration 5 :  0.577734239856099
volume iteration 5 :  0.577734239856099
cost at iteration 6 :  0.5422215841909237
volume iteration 6 :  0.5422215841909237
cost at iteration 7 :  0.5087794048707024
volume iteration 7 :  0.5087794048707024
cost at iteration 8 :  0.47736842676430696
volume iteration 8 :  0.47736842676430696
cost at iteration 9 :  0.4479208481072295
volume iteration 9 :  0.4479208481072295
cost at iteration 10 :  0.4203473018180518
volume iteration 10 :  0.4203473018180518


KeyboardInterrupt: 